# Demo of default functions df\_point\_append\_values

*****

__This script is the "official demo" of a function. Please if you want to modify it, work on your own copy__


The function takes a csv dataframe, a xarray.Dataset, the names of the longitude and latitude columns in the csv dataframe, the crs of the csv dataframe, and the shift value (in degrees) to be added to the csv dataframe longitude and latitude columns (in csv dataframe coordinates units)

**df**: the dataframe to which you want to append the xarray values  
**df_lon**: the name of the longitude column in your csv file  
**df_lat**: the name of the latitude column in your csv file  
**df_crs**: the coordinate reference system of the csv file  
**ds**: the xarray dataset to get values from  
**pts_shift**: the number of pixels to shift the csv points up and right , defaults to 0 (optional,
this option is usefull when for example points location correspond to the exact corner of the xarray
dataset (positive value are appropriate to lower-left corner)


Return a dataframe with the same number of rows as the input dataframe, and one column for each of
the variables in the xarray.Dataset.

Documentation for a given function can be accessed simply by adding ? at the end of the function in a cell. e.g. `df_point_append_values?` or by selecting the function and pressing `Shift-Tab`.

In [ ]:
# Make sure the script is using the proper kernel
try:
    %run ../swiss_utils/assert_env.py
except:
    %run ./swiss_utils/assert_env.py

In [ ]:
# Import modules

# reload module before executing code
%load_ext autoreload
%autoreload 2

# define modules locations (you might have to adapt define_mod_locs.py)
%run ../swiss_utils/define_mod_locs.py


import numpy as np
import pandas as pd
import xarray as xr

from datetime import datetime
from pyproj import Transformer
from itertools import product as iterprod

from swiss_utils.data_cube_utilities.sdc_utilities import load_multi_clean, write_geotiff_from_xr

import datacube
dc = datacube.Datacube()

import warnings
warnings.filterwarnings("ignore")

# AND THE FUNCTION
from swiss_utils.data_cube_utilities.sdc_utilities import optimize_types, df_point_append_values

## Load Landsat or Sentinel 2 product, compute NDVI TS statistics and get values

The next cell contains the dataset configuration information:
- product **should be Landsat or Sentinel 2**
- geographical extent
- time period
- bands **should contains at least ['red', 'nir'] plus the quality band**

You can generate it in three ways:
1. manually from scratch,
2. by manually copy/pasting the final cell content of the [config_tool](config_tool.ipynb) notebook,
3. by loading the final cell content of the [config_tool](config_tool.ipynb) notebook using the magic `%load config_cell.txt`.

In [ ]:
%load config_cell.txt

In [ ]:
# Load and clean the dataset

ds_clean, clean_mask = load_multi_clean(dc = dc, products = product, time = [start_date, end_date],
                                        lon = [min_lon, max_lon], lat = [min_lat, max_lat],
                                        measurements = measurements)
crs = ds_clean.crs
del clean_mask

In [ ]:
# Compute NDVI Time Serie statistics

ndvi = (ds_clean.nir - ds_clean.red) / (ds_clean.nir + ds_clean.red)
del ds_clean
ndvi = ndvi.where(np.isfinite(ndvi)) # replace +-Inf by nan
ds_stats = ndvi.mean(dim=['time'], skipna = True).to_dataset(name = 'ndvi_mean')
ds_stats = ds_stats.merge(ndvi.median(dim=['time'], skipna = True).to_dataset(name = 'ndvi_median'))
ds_stats = ds_stats.merge(ndvi.std(dim=['time'], skipna = True).to_dataset(name = 'ndvi_std'))
del ndvi
ds_stats.attrs['crs'] = crs

## Generate a virtual point grid (in EPSG:4326) within the dasat extent

In a real situation you can load point file, e.g.:
```Python
df_csv = pd.read_csv(csv_path, sep = ',')
df_csv = optimize_types(df_csv)
```

In [ ]:
# Generate a virtual point grid (in EPSG:4326)if you were note able to load a csv point

n_rows, n_cols = 50, 50
coords = list(iterprod(np.arange(min_lon, max_lon, (max_lon - min_lon) / (n_cols + 1))[1:],
                       np.arange(min_lat, max_lat, (max_lat - min_lat) / (n_rows + 1))[1:]))
df_csv = pd.DataFrame (coords, columns = ['lon', 'lat'])
csv_lon_var = 'lon'
csv_lat_var = 'lat'
csv_crs = 4326

## Get values from ds_stats

In [ ]:
df_csv = df_point_append_values(df_csv, csv_lon_var, csv_lat_var, csv_crs, ds_stats)
print(df_csv)

## Export output for check values

In [ ]:
df_csv.to_csv('df_csv.csv')

write_geotiff_from_xr(tif_path = 'ds_stats.tif',
                      dataset = ds_stats,
                      compr = 'DEFLATE')